<a href="https://colab.research.google.com/github/Ramadane735/CSS-AVANC-E/blob/main/Checkpoint_sur_le_Web_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

 # Question 1: Récupère le contenu HTML d'une page Wikipédia

def get_html(url):

    response = requests.get(url)
    response.raise_for_status()
    return BeautifulSoup(response.content, "html.parser")

 # Question 2: Extrait le titre de l'article

def get_title(soup):
    return soup.find("h1", id="firstHeading").text.strip()

 # Question 3: Extrait le contenu de l'article en structurant le texte par en-têtes

def get_article_content(soup):

    content = {}
    content_div = soup.find("div", id="mw-content-text")
    current_heading = "Introduction"
    content[current_heading] = ""

    for elem in content_div.find_all(['h2', 'p']):
        if elem.name == 'h2':
            span = elem.find('span', class_='mw-headline')
            if span:
                current_heading = span.text.strip()
                content[current_heading] = ""
        elif elem.name == 'p':
            text = elem.get_text(strip=True)
            if text:
                content[current_heading] += text + "\n"

    return content

 # Question 4: Collecte tous les liens internes menant à d'autres pages Wikipédia

def get_internal_links(soup, base_url="https://fr.wikipedia.org"):

    links = set()
    for a_tag in soup.find_all("a", href=True):
        href = a_tag['href']
        if href.startswith("/wiki/") and not any(prefix in href for prefix in [":", "#", "//"]):
            full_url = urljoin(base_url, href)
            links.add(full_url)
    return list(links)

 # Question 5: Fonction principale consolidée

def extract_wikipedia_data(url):

    soup = get_html(url)
    return {
        "url": url,
        "title": get_title(soup),
        "content": get_article_content(soup),
        "internal_links": get_internal_links(soup)
    }

 # Question 6: Exemple de test de la dernière fonction sur une page Wikipedia
if __name__ == "__main__":
    test_url = "https://fr.wikipedia.org/wiki/Intelligence_artificielle"
    data = extract_wikipedia_data(test_url)

    print("Titre de l'article :", data["title"])
    print("\nSections extraites :")
    for heading in data["content"].keys():
        print("-", heading)
    print(f"\nNombre de liens internes trouvés : {len(data['internal_links'])}")
    print("Premier lien interne :", data["internal_links"][0] if data["internal_links"] else "Aucun lien")


Titre de l'article : Intelligence artificielle

Sections extraites :
- Introduction

Nombre de liens internes trouvés : 1195
Premier lien interne : https://fr.wikipedia.org/wiki/Manipulation_boursi%C3%A8re
